# 크롤링

In [1]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request 
from selenium.webdriver import Chrome
import re     
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [2]:
def get_videos (channel): 
    
    # 브라우저 열기
    delay=3
    browser = Chrome()
    browser.implicitly_wait(delay)

    # url을 받아 창을 열고, 최대화
    start_url  = channel
    browser.get(start_url)  
    browser.maximize_window()

    # 영상목록 클릭 후 body 찾기
    body = browser.find_element_by_css_selector('body')

    
    # 5번 밑으로 내리기 -> 타임라인 업데이트를 위해서!
    num_of_pagedowns = 2
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1)
        num_of_pagedowns -= 1
    
    # 파싱 후 크롤링
    html0 = browser.page_source
    soup = BeautifulSoup(html0,'lxml')
    
    videos = soup.select('h3 > a')

    title = []
    url = []

    for video in videos:
        if video.get('href')[:7] != '/watch?':
            pass
        else:
            title.append(video.text)
            url.append('https://www.youtube.com/' + video.get('href'))
    
   

    browser.close() 
    return url, title


'https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww'

'https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww'

In [3]:
url_list, title_list = get_videos('https://www.youtube.com/channel/UC3HH4dZj3CZmrCC0MuDliww')

In [53]:
def get_info(title_list, url_list,cnt):

    df = pd.DataFrame({
        '날짜':[],
        '제목':[],
        '조회수':[],
        '댓글 수':[],
        'URL':[],
        '좋아요 수':[],
        '싫어요 수':[],
        '댓글' : []})

    for i in reversed(range(cnt)): # 최신 순으로 추출하기 위해
        name = title_list[i]
        start_url = url_list[i]
        
        browser = Chrome()
        browser.get(start_url)
        browser.maximize_window()
        
        body = browser.find_element_by_tag_name('body')
        num_page_down = 5 # 20번 이상이 적당함.
        while num_page_down:
            body.send_keys(Keys.PAGE_DOWN)
            time.sleep(1.5)
            num_page_down -= 1
            
            
        html0 = browser.page_source
        html = BeautifulSoup(html0,'html.parser')

        start_date = html.find('div',{'id':'info-strings','class':'style-scope ytd-video-primary-info-renderer'}).text
        start_date = re.sub(r'.', '', start_date, count = 1)
        
        view = html.find('span',{'class':'view-count style-scope ytd-video-view-count-renderer'}).text
        view = re.sub(r'.', '', view, count = 4)
        
        time.sleep(1)
        
        # 댓글 수
        try:
            comment_num = html.find('yt-formatted-string',
                                {'class' : "count-text style-scope ytd-comments-header-renderer"}).text
        except:
            pass

        #좋아요수
        likes_num = html.find('yt-formatted-string',{'id':'text',
                                                     'class':'style-scope ytd-toggle-button-renderer style-text',
                                                     'aria-label':re.compile('좋아요')}).text+'개'


        #싫어요수
        unlikes_num = html.find('yt-formatted-string',{'id':'text',
                                                       'class':'style-scope ytd-toggle-button-renderer style-text',
                                                       'aria-label':re.compile('싫어요')}).text+'개'
        
        # 댓글
        comment = []
        comment_data = html.find_all('ytd-comment-renderer',
                           {'class':'style-scope ytd-comment-thread-renderer'})
        
        for i in range(len(comment_data)):
            comments = comment_data[i].find('yt-formatted-string',
                                       {'id':'content-text','class':'style-scope ytd-comment-renderer'}).text

            try:
                tmp1 = comment_data[i].find('span',{'id':'vote-count-left'}).text
                like = "".join(re.findall('[0-9]',tmp1)) + "개"
            except:
                like = 0

            tmp2 = comment_data[i].find('a',{'id':'author-text'}).find('span').text
            youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',tmp2))

            comment.append([youtube_id, comments, like])
            
            
        insert_data = pd.DataFrame({
        '날짜':[start_date],
        '제목':[name],
        '조회수':[view],
        '댓글 수':[comment_num],
        'URL':[start_url],
        '좋아요 수':[likes_num],
        '싫어요 수':[unlikes_num],
        '댓글' : [comment]})

        df = df.append(insert_data)
        browser.quit()
        
    return df


In [54]:
df = get_info(title_list, url_list,2)
df

,날짜,제목,조회수,댓글 수,URL,좋아요 수,싫어요 수,댓글
0,021. 7. 31.,뉴욕 한복판에 떨어진 초거대 괴물 1부,"91,463회",댓글 175개,https://www.youtube.com//watch?v=V3mYegiLnVA&p...,1천개,23개,"[[Kevinkim, 진짜 골목에서 군인들이 괴물 공격하는 장면은...진짜 어릴 때..."
0,021. 7. 31.,뉴욕 한복판에 떨어진 초거대 괴물 2부,"56,058회",댓글 155개,https://www.youtube.com//watch?v=rBqy-LJZTT4&p...,657개,35개,"[[코주부육포, 제일 불친절한 시리즈 ㅋㅋㅋㅋㅋ떡밥이 속 시원하게 풀리는 일이 없음..."


In [60]:
def get_video():
    feature = input('검색어를 입력하시오 : ')
    
    driver = Chrome()
    driver.get('https://www.youtube.com')
    
    n = 3
    while n > 0:
        print('웹페이지를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
        
    src = driver.find_element_by_xpath('//*[@id="search"]')
    src.send_keys(feature)
    src.send_keys(Keys.RETURN)
    
    n = 2
    while n > 0:
        print('검색 결과를 불러오는 중입니다..' + '..' * n)
        time.sleep(1)
        n -= 1
        
    print('데이터 수집 중입니다....')
               
    html = driver.page_source
    soup = BeautifulSoup(html)
    
    df_title = []
    df_link = []
    df_writer = []
    df_view = []
    df_date = []
    
    for i in range(len(soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated'))):
        title = soup.find_all('a', {'id' : 'video-title'})[i].text.replace('\n', '')
        link = 'https://www.youtube.com/' + soup.find_all('a', {'id' : 'video-title'})[i]['href']
        writer = soup.find_all('ytd-channel-name', 'long-byline style-scope ytd-video-renderer')[i].text.replace('\n', '').split(' ')[0]
        view = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[3]
        date = soup.find_all('ytd-video-meta-block', 'style-scope ytd-video-renderer byline-separated')[i].text.split('•')[1].split('\n')[4]
    
        df_title.append(title)
        df_link.append(link)
        df_writer.append(writer)
        df_view.append(view)
        df_date.append(date)
          
    df_just_video = pd.DataFrame(columns=['영상제목', '채널명', '영상url', '조회수', '영상등록날짜'])

    df_just_video['영상제목'] = df_title
    df_just_video['채널명'] = df_writer
    df_just_video['영상url'] = df_link
    df_just_video['조회수'] = df_view
    df_just_video['영상등록날짜'] = df_date
          
    df_just_video.to_csv('df_just_video.csv', encoding='utf-8-sig', index=False)
    
    driver.close()
          
    result = input('데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)')
    if result == 'y':
        display(df_just_video)
        question = input('원하는 영상을 재생하시겠습니까? (y/n)')
        if question == 'y':
            button = int(input('재생하고자 하는 영상의 번호(출력된 표 가장 왼쪽의 번호)를 입력해주세요.'))
            driver = webdriver.Chrome(path)
            driver.get(df_just_video['영상url'][button])
        else:
            return '프로그램을 종료합니다.'
    else:
        return '프로그램을 종료합니다.'

In [61]:
get_video()

검색어를 입력하시오 : 영화
웹페이지를 불러오는 중입니다........
웹페이지를 불러오는 중입니다......
웹페이지를 불러오는 중입니다....
검색 결과를 불러오는 중입니다......
검색 결과를 불러오는 중입니다....
데이터 수집 중입니다....


c:\users\samsung\appdata\local\programs\python\python37\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file c:\users\samsung\appdata\local\programs\python\python37\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


데이터프레임 저장이 완료되었습니다! 데이터프레임을 조회하시겠습니까? (y/n)y


,영상제목,채널명,영상url,조회수,영상등록날짜
0,러시아에서 만든 레전드 1위 '다크 판타지' 액션!! [영화리뷰 결말포함],동네비디오방,https://www.youtube.com//watch?v=Vf9Nnyux9dc,조회수 2.4천회,3시간 전
1,자막) 전 세계에서 흥행 수익 3천만 달러를 기록한 올해의 공포 영화 (영화리뷰 결...,영화대장,https://www.youtube.com//watch?v=wMHj1_nEWEg,조회수 2.4만회,20시간 전
2,노숙자에게 친절을 베풀고 충격적인 일을 당한 가족 - 영화리뷰 결말포함,MOBEE,https://www.youtube.com//watch?v=sXxHEV900Lc,조회수 173회,47분 전
3,탈출불가의 소말리아에서 한국인의 기발함으로 탈출한 실화기반 화제의 신작영화 [모가디슈],고몽,https://www.youtube.com//watch?v=4IYpOzxpHaI,조회수 186만회,2개월 전
4,너무 충격적 실화였기에 개봉자체를 중국에서 끝까지 막고자 했던 영화,제이나나,https://www.youtube.com//watch?v=R-bsMpQMX7M,조회수 14만회,2일 전
5,불법 지하 격투장을 혼자서 쓸어버리는 역대 최강 누님 [영화리뷰 결말포함],동네비디오방,https://www.youtube.com//watch?v=6RsBVAa2OsE,조회수 2.1만회,19시간 전
6,남자들이 보면 피가 끓는다는 고대 콜로세움 영화 수준이 ㅎㄷㄷ...,무비타이저(Movie,https://www.youtube.com//watch?v=zQcS4hfKwOk,조회수 8.1만회,5일 전
7,하.. 넷플릭스가 미쳤나 333 (전세계 시청 순위 1위 ㄷㄷ...),김시선,https://www.youtube.com//watch?v=x0GYcdXgaFA,조회수 1.1만회,2일 전
8,세상에 이런 영화가 있을 줄이야... 진짜 보시면 난리납니다ㅋㅋㅋㅋㅋ [영화리뷰 /...,결말남포함녀,https://www.youtube.com//watch?v=xBboy4vbY-I,조회수 7.3만회,4일 전
9,약빤 사람이 리메이크한 영화 300 [병맛영화][결말포함],배우는,https://www.youtube.com//watch?v=THVromjGNBw,조회수 115만회,4개월 전


원하는 영상을 재생하시겠습니까? (y/n)n


'프로그램을 종료합니다.'